# Florence-2-large sample usage

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/flourence_dataset
%cd    /content/drive/MyDrive/flourence_dataset
!mkdir /content/drive/MyDrive/flourence_dataset/dataset

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/MyDrive/flourence_dataset’: File exists
/content/drive/MyDrive/flourence_dataset
mkdir: cannot create directory ‘/content/drive/MyDrive/flourence_dataset/dataset’: File exists


In [2]:
!pip install flash_attn einops timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cuspa

In [3]:
from transformers import AutoProcessor, AutoModelForCausalLM
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import requests
import copy
import cv2
import os
%matplotlib inline

In [4]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## define the prediction function

In [ ]:
video_files = os.listdir()[1:-3]
video_files

['D17_S20231204180000_E20231204180800.mp4',
 'D17_S20231204105314_E20231204111124.mp4',
 'D17-S20231204180000-E20231204180.mp4',
 'D34-S20231205085500-E20231205090.mp4',
 'WhatsApp Video 2023-12-04 at 4.15.42 PM.mp4',
 'WhatsApp Video 2023-12-04 at 4.16.12 PM.mp4',
 'WhatsApp Video 2023-12-04 at 4.16.23 PM.mp4',
 'D17_S20231204103503_E20231204105314.mp4',
 'D17_S20231204105314_E20231204111124 (video-converter.com).mp4']

In [ ]:
for video_path in video_files:

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print(fps)

25
25
25
30
25
25
25
25
25


In [5]:
def run_example(image, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [6]:
def plot_bbox(image, data = None):
   # Create a figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    if data != None:
      # Plot each bounding box
      for bbox, label in zip(data['bboxes'], data['labels']):
          # Unpack the bounding box coordinates
          x1, y1, x2, y2 = bbox
          # Create a Rectangle patch
          rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
          # Add the rectangle to the Axes
          ax.add_patch(rect)
          # Annotate the label
          plt.text(x1, y1, label, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    # Remove the axis ticks and labels
    ax.axis('off')

    # Show the plot
    plt.show()

In [7]:
 categories = {
        'person':{
            "id": 0,
            "name": "person",
            "supercategory": "none"
        },
        'cap':{
            "id": 1,
            "name": "cap",
            "supercategory": "Personal-Equipments"
        },
        'gloves':{
            "id": 2,
            "name": "gloves",
            "supercategory": "Personal-Equipments"
        },
        'boots':{
            "id": 3,
            "name": "boots",
            "supercategory": "Personal-Equipments"
        },
}

In [8]:
def create_dataframe():
  image_df =  pd.DataFrame(columns = ["id", "file_name", "height", "width"])
  label_df = pd.DataFrame(columns = ["id", "image_id", "category_id", "bbox","area"])
  return image_df , label_df

def add_row(df, row):
  df.loc[len(df)] = row

def create_csv(df, file_name):
  df.to_csv(file_name, sep = "|")

def append_csv(df, file_name):
  df.to_csv(file_name, mode = "a", index = False, sep = "|", header=False)

In [ ]:
# do not run every time
def initiate_video_dataframe():
  image_df , label_df = create_dataframe()
  create_csv(image_df, "image.csv")
  create_csv(label_df, "label.csv")
  return image_df, label_df

image_df, label_df = initiate_video_dataframe()

In [9]:
def save_image(image, name):
  image.save("/content/drive/MyDrive/flourence_dataset/dataset/"+name)

In [10]:
sample_rate = 1
image_id = 2826
annotate_id = 9238
output_dir = '/dataset'
prompt = "person, cap, gloves, boots"
task_prompt = '<CAPTION_TO_PHRASE_GROUNDING>'
image_df, label_df = create_dataframe()

def create_dataset(video_path):
    global image_id, annotate_id
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frame rate of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = fps // sample_rate

    frame_count = 0

    while True:
        ret, frame = cap.read()

        # If the frame was not read successfully, break the loop
        if not ret:
            break

        # Save the frame at the specified interval
        if frame_count % frame_interval == 0:

            image = Image.fromarray(frame)
            plot_bbox(image)
            results = run_example(image, task_prompt, text_input=prompt)
            plot_bbox(image, results[task_prompt])

            bboxes = results[task_prompt]['bboxes']
            labels = results[task_prompt]['labels']

            if not labels:
              frame_count += 1
              continue

            frame_name = f"image_{image_id}.jpg"
            height = image.height
            width = image.width

            add_row(image_df, [image_id, frame_name, height, width])
            display(image_df.tail())
            save_image(image, frame_name)

            for i in range(len(labels)):
              x0, y0, x1, y1 = bboxes[i]
              label = labels[i]
              cat_id = categories[label]['id']
              height = y1 - y0
              width = x1 - x0
              area = height * width
              add_row(label_df, [annotate_id, image_id, cat_id, [x0, y0, width, height], area])
              annotate_id += 1
            display(label_df.tail())
            image_id += 1
        frame_count += 1

    cap.release()
    append_csv(image_df, "image.csv")
    append_csv(label_df, "label.csv")
    print(image_id, annotate_id)



In [ ]:
create_dataset('/content/drive/MyDrive/flourence_dataset/6min part2.mp4')